In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
import pandas as pd
import numpy as np
import pickle

In [5]:
def pkl_process(data=None, file=None, mode="dump"):
    f_mode = "wb" if mode == "dump" else "rb"
    load_data = None
    
    with open(file, f_mode) as f:
        if mode == "dump":
            pickle.dump(data, f)
        else:
            load_data = pickle.load(f)
    
    return load_data

# load categorized data and split train, test

In [19]:
df = pd.read_csv("categorized_NAFLD.csv")
df = df.drop(columns=['Unnamed: 0'])
df.shape

(488, 32)

In [20]:
df.head()

,NGT_IFG_,NGT__PRE,DM_STATU,MS,AGE,GNDR,ETHNICIT,BMI,HOMA,SBP,...,AST_M0,ALT_M0,BILIRRUB,PLATELET,ALBUMIN,TSH,MEAN_FPG,NASH_OU2,FIBROSI2,NAFLD_OU
0,0,0,1,0,0,0,0,2,0,1,...,0,0,0,1,1,0,2,0,0,0
1,1,2,0,1,2,1,2,3,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,0,1,1,2,1,2,2,0,0,...,0,0,0,1,1,0,1,0,0,0
3,0,0,1,0,1,0,0,3,1,1,...,0,0,0,1,1,0,2,0,0,0
4,0,0,1,1,3,1,0,2,1,1,...,0,0,0,1,1,0,2,0,0,0


In [46]:
# split the data fram to train and test and shuffle
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=43)
train_df.shape, test_df.shape, len(train_df[train_df['NAFLD_OU'] == 1]), len(train_df[train_df['NAFLD_OU'] == 0]), len(test_df[test_df['NAFLD_OU'] == 1]), len(test_df[test_df['NAFLD_OU'] == 0])

((390, 32), (98, 32), 274, 116, 68, 30)

In [47]:
train_df.head()

,NGT_IFG_,NGT__PRE,DM_STATU,MS,AGE,GNDR,ETHNICIT,BMI,HOMA,SBP,...,AST_M0,ALT_M0,BILIRRUB,PLATELET,ALBUMIN,TSH,MEAN_FPG,NASH_OU2,FIBROSI2,NAFLD_OU
235,1,2,0,0,0,1,3,2,0,0,...,0,1,0,1,1,0,1,0,0,1
234,2,2,0,1,1,0,2,3,1,0,...,1,1,0,1,0,0,1,1,1,1
458,0,0,1,1,1,0,3,2,0,1,...,1,0,0,1,1,0,1,1,1,1
18,0,0,1,0,2,1,0,3,1,1,...,0,0,0,1,0,0,0,0,0,0
121,0,0,1,0,2,1,2,3,2,1,...,0,0,0,0,1,0,2,0,0,0


In [49]:
#get training y label
train_y_nash, train_y_fib, train_y_nafld = np.array(train_df['NASH_OU2']), np.array(train_df['FIBROSI2']), np.array(train_df['NAFLD_OU'])
train_y_nafld[:5]

array([1, 1, 1, 0, 0])

In [53]:
train_X = train_df.drop(columns=['NASH_OU2', 'FIBROSI2', 'NAFLD_OU'])
train_X.shape, train_y_nafld.shape

((390, 29), (390,))

In [54]:
test_y_nash, test_y_fib, test_y_nafld = np.array(test_df['NASH_OU2']), np.array(test_df['FIBROSI2']), np.array(test_df['NAFLD_OU'])
test_X = test_df.drop(columns=['NASH_OU2', 'FIBROSI2', 'NAFLD_OU'])
test_X.shape, test_y_nafld.shape

((98, 29), (98,))

# data encoding

## one-hot encoding

In [11]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [21]:
ohe.fit_transform().toarray().shape

(488, 79)

## feature hashing

In [ ]:
from sklearn.feature_extraction import FeatureHasher
#convert data to uni-feature space then feature hashing (libsvm format first -> convert to dict)

# machine learning 

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, precision_recall_fscore_support

## Logistic Regression

### label encoding as features

###  OHE as features

### feature hashing as features

### cross-validation

In [ ]:
tuned_parameters = {'max_iter':[100, 500, 1000], 'tol':[0.0001, 0.001, 0.01, 0.1],'random_state':[13], 'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0], 'solver':['lbfgs', 'liblinear', 'newton-cg']}
gs = GridSearchCV(LogisticRegression(), tuned_parameters, scoring='roc_auc', n_jobs=-1, cv=5) #scoring='roc_auc'
gs.fit(lr_train_X, lr_train_y)
print(gs.best_params_)
print(gs.best_estimator_)

### train and predict

In [ ]:
clf = gs.best_estimator_
clf.fit(train_data, train_label)

### evaluation

#### auc_roc

In [ ]:
def get_cutoff():
    pass

In [ ]:
print(roc_auc_score(test_label, pred_result_1))
fpr, tpr, thresholds = roc_curve(test_label, pred_result_1)
plt.plot(fpr, tpr, lw=1, alpha=0.5)
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Chance', alpha=.8)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

#### precision, recall, f1-score

## SVM

## random forest

## XGBoost

# feature elimination